In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import pylab as pl

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [3]:
## Load the data

transform = transforms.Compose([transforms.ToTensor(), 
                                torchvision.transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = tuple(range(10))

HTTPError: HTTP Error 503: Service Unavailable

In [51]:
## Define the neural net

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # Convolution layers
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        
        # Affine operations Ax+b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Convolutions + max pooling over a 2x2 window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        # Affine transformations
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # Discard the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features

In [221]:
## Train the neural net

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
n_print = 200
for epoch in range(3):
    running_loss = 0.
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % n_print == n_print-1:
            print("[{}, {}] loss: {}".format(epoch+1, i+1, running_loss/n_print))
            running_loss = 0.
        running_loss = 0.
            
print("Finished training")

[1, 200] loss: 0.00173961341381073
[1, 400] loss: 0.0022546304762363434
[1, 600] loss: 0.0009170521795749665
[1, 800] loss: 0.0004688012972474098
[1, 1000] loss: 0.0006367313116788864
[1, 1200] loss: 0.0005606236308813095
[1, 1400] loss: 0.0004680261015892029
[1, 1600] loss: 3.516363445669413e-05
[1, 1800] loss: 0.0020598261058330535
[2, 200] loss: 0.00017335735261440278
[2, 400] loss: 0.0014783193171024323
[2, 600] loss: 0.00037808820605278015
[2, 800] loss: 0.0010314495116472244
[2, 1000] loss: 0.000377642847597599
[2, 1200] loss: 0.0002714717574417591
[2, 1400] loss: 1.964168157428503e-05
[2, 1600] loss: 6.087426096200943e-05
[2, 1800] loss: 0.0001664086803793907
[3, 200] loss: 0.00047371715307235716
[3, 400] loss: 1.870901440270245e-05
[3, 600] loss: 9.023058228194714e-05
[3, 800] loss: 0.00013943318277597427
[3, 1000] loss: 1.7886969726532697e-05
[3, 1200] loss: 2.2716980893164874e-05
[3, 1400] loss: 0.00013994721695780753
[3, 1600] loss: 0.0003911813721060753
[3, 1800] loss: 0.00

In [225]:
## Misclassification error on test set

misclassified = 0
for i, data in enumerate(testloader, 0):
    images, labels = data
    out = net(images)
    misclassified += torch.sum(torch.max(out, 1).indices != labels)

(misclassified/len(testset)).item()

0.01759999990463257

In [226]:
## Misclassification error on train set

misclassified = 0
for i, data in enumerate(trainloader, 0):
    images, labels = data
    out = net(images)
    misclassified += torch.sum(torch.max(out, 1).indices != labels)

(misclassified/len(trainset)).item()

0.015716666355729103